<a href="https://colab.research.google.com/github/wksmirnowa/antidict/blob/master/Jupyter%20Notebooks/Expressive_forms_classification_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 59.7MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [0]:
import pandas as pd

In [3]:
data = pd.read_csv('expressive_words_train.txt', names=['words'])
data['words'] = data.words.str.lower()
data['values'] = 1
print(data.sample(20))

              words  values
16       заколдовка       1
186       мальчичек       1
59       ольгушечка       1
209     ксилофонщик       1
243          маманя       1
263      девчушечка       1
22         солнышко       1
352          кисусь       1
84            димка       1
333       стиксянам       1
135         андрюша       1
359    сестробратцы       1
184          дедуля       1
423     легохонькие       1
320        репицька       1
356  полуторолеткой       1
310          клунок       1
56         олюньчик       1
343      печененьки       1
150         букетик       1


In [4]:
data1 = pd.read_csv('zaliznyak.txt', names=['words'])
data1['values'] = 0
print(data1.sample(20))

                  words  values
803       перфорируемое       0
2797        онемевавшею       0
1508       воссоединишь       0
1172          нацарапал       0
2058      взобравшегося       0
1073      замусливающую       0
166          углубившей       0
531       отскакиваемую       0
980         расщепаться       0
22    обстрагивавшегося       0
77              щебенил       0
2345    реставрационной       0
94          еретическом       0
1175      разжёвывается       0
2346     обандеролившее       0
1824         перистилем       0
953       водружавшейся       0
171     переучитываемую       0
2055      затрагивающую       0
1397        отклонилась       0


In [37]:
len(data1)

2955

In [0]:
import gensim
import numpy as np

from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [6]:
!wget https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz

--2020-05-17 16:52:06--  https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691248108 (2.5G) [application/x-gzip]
Saving to: ‘araneum_none_fasttextcbow_300_5_2018.tgz’

araneum_none_fastte 100%[===================>]   2.51G  28.1MB/s    in 93s     

2020-05-17 16:53:40 (27.5 MB/s) - ‘araneum_none_fasttextcbow_300_5_2018.tgz’ saved [2691248108/2691248108]



In [0]:
import tarfile
tar = tarfile.open("araneum_none_fasttextcbow_300_5_2018.tgz", "r")
tar.extractall()

In [0]:
model = gensim.models.KeyedVectors.load("araneum_none_fasttextcbow_300_5_2018.model")

In [46]:
df = pd.concat([data1.sample(600), data], axis=0).sample(frac=1)
print(df.sample(20))

                     words  values
234                зверюга       1
1284              выстужен       0
605            камвольными       0
1440           подкорнаюсь       0
1487         непристойными       0
35            обсаживаемом       0
1492  набронзировывающийся       0
721              винтуемою       0
186              мальчичек       1
715          понадивлённее       0
272           уточняющеюся       0
285                пивасик       1
2030         проколупанное       0
1329     корригировавшеюся       0
255             девчоночка       1
40              взломаться       0
339            криволапках       1
234          притаптываемо       0
382            тунтунколка       1
1389           свистнутого       0


In [0]:
df['model'] = df.words.apply(lambda x: model[x])

In [0]:
X = np.vstack(df.model.values)
y = df['values'].values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
clf = LogisticRegression().fit(X_train, y_train)

In [51]:
y_pred = clf.predict(X_test)
f1_score(y_test, y_pred)

0.7624309392265193

In [15]:
from sklearn import linear_model
clf1 = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf1.fit(X, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [16]:
y_pred = clf1.predict(X_test)
f1_score(y_test, y_pred)

0.9035087719298246

In [0]:
dataset = pd.read_csv('words_only.csv', sep='\t', names=['word'])

In [53]:
%%time
ff = open('expressive1.csv', 'w+')

for word in dataset.word:
    #if len(word) < 4 or len(word) > 18:
      #continue
    res = clf1.predict(model[word].reshape(1,-1))
    if res[0] == 1:
        example = f"{word}\t{res}" + "\n"
        ff.write(example)

CPU times: user 16min 34s, sys: 779 ms, total: 16min 34s
Wall time: 16min 34s


In [62]:
examples1 = pd.read_csv('expressive1.csv', sep='\t', names=['word', 'class'])
examples1[-101060:-101000]

,word,class
704181,тренажераха,[1]
704182,тренажерахи,[1]
704183,тренажерики,[1]
704184,тренажерить,[1]
704185,тренажеркам,[1]
704186,тренажерках,[1]
704187,тренажеркою,[1]
704188,тренажернах,[1]
704189,тренажернвй,[1]
704190,тренажерник,[1]


In [58]:
len(examples1)

805241

In [0]:
dataset = pd.read_csv('words_only (2).csv', sep='\t', names=['word'])

In [20]:
%%time
ff = open('expressive.csv', 'w+')

for word in dataset.word:
    if len(word) < 4 or len(word) > 18:
      continue
    res = clf1.predict(model[word].reshape(1,-1))
    if res[0] == 1:
        example = f"{word}\t{res}" + "\n"
        ff.write(example)

CPU times: user 16min 41s, sys: 721 ms, total: 16min 42s
Wall time: 16min 42s


In [76]:
#examples = pd.read_csv('expressive.csv', sep='\t', names=['word', 'class'])
examples.sample(15)

,word,class
724186,ляжечек,[1]
521344,виндень,[1]
783289,напотагама,[1]
779894,навичо,[1]
290631,пентакса,[1]
175903,коколико,[1]
504268,быдлястые,[1]
978822,туснике,[1]
108915,дзавальей,[1]
792231,непикапер,[1]


In [22]:
len(examples)

1071964